In [1]:
import os
import nibabel as nib
import imageio
import numpy as np
from tqdm import tqdm


In [2]:
def nii_to_image(baseDir,objDir):
    files = os.listdir(baseDir)  # 读取nii文件夹
    with tqdm(total = len(files)) as pbar:
        for file in files:
            if  ".nii.gz" not in file:
                continue
            img_path = os.path.join(baseDir, file)
            img = nib.load(img_path)  # 读取nii
            img_fdata = img.get_fdata(dtype=np.float32)  # 读取为float32
            img_fdata = np.rot90(img_fdata)
            img_fdata = (img_fdata * 255).astype(np.uint8)
            fname = file.replace('.nii.gz', '')  # 去掉nii的后缀名
            img_f_path = os.path.join(objDir, fname)
            # 创建nii对应的图像的文件夹
            if not os.path.exists(img_f_path):
                os.mkdir(img_f_path)  # 新建文件夹
            
            # 将归一化的浮点数数据转换为8位整数数据
 
            # 开始转换为图像
            (x, y, z) = img.shape
            for i in range(z):  # z是图像的序列
                slice = img_fdata[:, :, i]  # 选择哪个方向的切片都可以
                # 保存图像，使用PNG格式
                imageio.imwrite(os.path.join(img_f_path, '{}.png'.format(i)), slice)
            pbar.update(1)
 


def normalize(slice):
    """Normalize the image slice to [0, 255]"""
    min_val = np.min(slice)
    max_val = np.max(slice)
    if max_val - min_val > 0:
        return ((slice - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    else:
        return np.zeros_like(slice, dtype=np.uint8)

In [3]:
baseDir = r"/home/konata/Dataset/TED_MRI/T2/mask/origin/"
objDir= r"/home/konata/Dataset/TED_MRI/T2/mask/origin_slice/TED_png/"
if not os.path.exists(objDir):
    os.mkdir(objDir)  # 新建文件夹
nii_to_image(baseDir,objDir)

100%|██████████| 126/126 [00:19<00:00,  6.51it/s]
